<a href="https://colab.research.google.com/github/Katsushi-Nakanishi/quant-macro/blob/main/GrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd
import numpy as np

pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

In [ ]:
['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

**・Lを総労働時間とした場合**

In [128]:
#対象の国をリスト化
target_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

#リストに該当する国の、1960~2000期間のデータを抽出
data = pwt90[
    pwt90['country'].isin(target_countries) &
    pwt90['year'].between(1960, 2000)
]

#必要な変数だけをリスト化し、データを絞る
relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh',]
data = data[relevant_cols].dropna()


data['alpha'] = 1 - data['labsh']  # 資本所得分配率（α）= 1 - 労働所得分配率
data['L'] = data['emp'] * data['avh']  # L（総労働時間） = emp * avh
data['y'] = data['rgdpna'] / data['L']  # y = Y/L
data['k'] = data['rkna'] / data['L']  # k = K/L


data = data.sort_values('year')  # 年順にソート
data = data.groupby('countrycode').apply(lambda x:x.assign(
    y_shifted = 100 * x['y'] / x['y'].iloc[0],
    k_shifted = 100 * x['k'] / x['k'].iloc[0]
)).reset_index(drop=True).dropna()  # 国別に、最初の年で基準化したy,k列をデータに追加


def calg(country_data):  #国別のデータに対して、成長会計をおこなう関数を定義

    start_year_actual = country_data['year'].min()  # 初年度
    end_year_actual = country_data['year'].max()  # 終年度

    start_data = country_data[country_data['year'] == start_year_actual].iloc[0]
    end_data = country_data[country_data['year'] == end_year_actual].iloc[0]

    years = end_data['year'] - start_data['year']

    g_y = ((end_data['y'] / start_data['y']) ** (1/years) - 1) * 100  # Growth Rate(労働生産性の伸び)

    g_k = ((end_data['k'] / start_data['k']) ** (1/years) - 1) * 100  # kの成長率を計算

    alpha_avg = (start_data['alpha'] + end_data['alpha']) / 2.0  # alphaの平均値を計算

    capital_deepening_contrib = alpha_avg * g_k  # Captal Deepning
    tfp_growth_contrib = g_y - capital_deepening_contrib  # TFP Growth

    tfp_share = tfp_growth_contrib / g_y
    cap_share = capital_deepening_contrib / g_y

    # 計算結果の辞書を返す
    return {
        'Country': start_data['country'],
        'Growth Rate': round(g_y, 2),
        'TFP Growth': round(tfp_growth_contrib, 2),
        'Capital Deepening': round(capital_deepening_contrib, 2),
        'TFP Share': round(tfp_share, 2),
        'Capital Share': round(cap_share, 2)
    }

# 国ごとにcalg関数を適用し、結果を辞書のリストとして作成
results_list = data.groupby('countrycode').apply(calg).dropna().tolist()
results_df = pd.DataFrame(results_list)
results_df

<ipython-input-128-1d60f4e73783>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('countrycode').apply(lambda x:x.assign(
<ipython-input-128-1d60f4e73783>:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_list = data.groupby('countrycode').apply(calg).dropna().tolist()


,Country,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
0,Australia,1.91,1.26,0.65,0.66,0.34
1,Austria,3.43,1.86,1.57,0.54,0.46
2,Belgium,3.30,2.09,1.21,0.63,0.37
3,Canada,1.86,1.11,0.75,0.59,0.41
4,Switzerland,1.75,0.71,1.04,0.41,0.59
5,Germany,3.78,2.22,1.56,0.59,0.41
6,Denmark,2.99,1.54,1.45,0.51,0.49
7,Spain,3.78,2.04,1.74,0.54,0.46
8,Finland,3.81,2.29,1.52,0.60,0.40
9,France,3.78,2.38,1.40,0.63,0.37


**・Lを就業者数とした場合**

In [129]:
#対象の国をリスト化
target_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

#リストに該当する国の、1960~2000期間のデータを抽出
data = pwt90[
    pwt90['country'].isin(target_countries) &
    pwt90['year'].between(1960, 2000)
]

#必要な変数だけをリスト化し、データを絞る
relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh',]
data = data[relevant_cols].dropna()


data['alpha'] = 1 - data['labsh']  # 資本所得分配率（α）= 1 - 労働所得分配率
data['y'] = data['rgdpna'] / data['emp']  # y = Y/emp
data['k'] = data['rkna'] / data['emp']  # k = K/emp


data = data.sort_values('year')  # 年順にソート
data = data.groupby('countrycode').apply(lambda x:x.assign(
    y_shifted = 100 * x['y'] / x['y'].iloc[0],
    k_shifted = 100 * x['k'] / x['k'].iloc[0]
)).reset_index(drop=True).dropna()  # 国別に、最初の年で基準化したy,k列をデータに追加


def calg(country_data):  #国別のデータに対して、成長会計をおこなう関数を定義

    start_year_actual = country_data['year'].min()  # 初年度
    end_year_actual = country_data['year'].max()  # 終年度

    start_data = country_data[country_data['year'] == start_year_actual].iloc[0]
    end_data = country_data[country_data['year'] == end_year_actual].iloc[0]

    years = end_data['year'] - start_data['year']

    g_y = ((end_data['y'] / start_data['y']) ** (1/years) - 1) * 100  # Growth Rate(労働生産性の伸び)

    g_k = ((end_data['k'] / start_data['k']) ** (1/years) - 1) * 100  # kの成長率を計算

    alpha_avg = country_data['alpha'].mean()  # alphaの平均値を計算

    capital_deepening_contrib = alpha_avg * g_k  # Captal Deepning
    tfp_growth_contrib = g_y - capital_deepening_contrib  # TFP Growth

    tfp_share = tfp_growth_contrib / g_y
    cap_share = capital_deepening_contrib / g_y

    # 計算結果の辞書を返す
    return {
        'Country': start_data['country'],
        'Growth Rate': round(g_y, 2),
        'TFP Growth': round(tfp_growth_contrib, 2),
        'Capital Deepening': round(capital_deepening_contrib, 2),
        'TFP Share': round(tfp_share, 2),
        'Capital Share': round(cap_share, 2)
    }

# 国ごとにcalg関数を適用し、結果を辞書のリストとして作成
results_list = data.groupby('countrycode').apply(calg).dropna().tolist()
results_df = pd.DataFrame(results_list)
results_df

<ipython-input-129-2dd0a76a53db>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('countrycode').apply(lambda x:x.assign(
<ipython-input-129-2dd0a76a53db>:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_list = data.groupby('countrycode').apply(calg).dropna().tolist()


,Country,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
0,Australia,1.74,1.15,0.59,0.66,0.34
1,Austria,3.08,1.64,1.44,0.53,0.47
2,Belgium,2.72,1.72,1.00,0.63,0.37
3,Canada,1.45,0.83,0.62,0.57,0.43
4,Switzerland,1.40,0.48,0.92,0.34,0.66
5,Germany,2.73,1.52,1.21,0.56,0.44
6,Denmark,2.31,1.10,1.21,0.48,0.52
7,Spain,3.50,1.86,1.63,0.53,0.47
8,Finland,3.38,2.01,1.37,0.59,0.41
9,France,2.86,1.75,1.12,0.61,0.39


In [125]:
print(data)

   countrycode    country  year         rgdpna          rkna        pop  \
0          AUS  Australia  1960  174210.734375  5.488126e+05  10.561457   
1          AUS  Australia  1961  176586.343750  5.626911e+05  10.803588   
2          AUS  Australia  1962  187672.468750  5.850144e+05  11.003531   
3          AUS  Australia  1963  200342.343750  6.088022e+05  11.214037   
4          AUS  Australia  1964  213012.218750  6.400842e+05  11.435046   
5          AUS  Australia  1965  217763.421875  6.679079e+05  11.660398   
6          AUS  Australia  1966  231225.156250  7.022682e+05  11.880264   
7          AUS  Australia  1967  243103.171875  7.355953e+05  12.101641   
8          AUS  Australia  1968  260524.234375  7.763284e+05  12.335469   
9          AUS  Australia  1969  279529.031250  8.186658e+05  12.618559   
10         AUS  Australia  1970  290615.187500  8.587118e+05  12.904760   
11         AUS  Australia  1971  301984.031250  8.994501e+05  13.150591   
12         AUS  Australia